<a href="https://colab.research.google.com/github/jrsheffie/Text-Sentimization/blob/JS_POC/POC_model_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')

#printing the stopwords in english
print(stopwords.words('english'))


#Loading the data from a csv file

X_data = pd.read_csv("/content/x_en_dataset.csv",encoding = 'ISO-8859-1' )
#facebook_data
#Instagram_Data
#Checking number of rows and collumns
X_data.shape

#print first 5 rows
X_data.head()

#Counting the number of missing values in the dataset
X_data.isnull().sum()

#Checking the dsitribution of the target column
X_data['sentiment'].value_counts()



['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,count
sentiment,
0.14,71
0.17,65
0.07,65
0.18,61
0.04,60
...,...
0.89,2
-0.93,1
0.93,1


In [ ]:


#Convert the main emotion Neutral to 1
X_data.replace({'main_emotion':{'neutral':0}},inplace=True)
#Convert the happy emotion to 1
X_data.replace({'main_emotion':['admiration','approval','caring','curiosity','desire','excitment','gratitude','joy','love','optimism','pride','realization','surprise']}, 1,inplace=True)
#Convert the negative emotion to -1
X_data.replace({'main_emotion':['anger','annoyance','confusion','disappointment','disapproval','disgust','embarrassment','excitement','fear','nervousness','remorse','sadness']},-1,inplace=True)


# Convert the column to string type to avoid TypeError
X_data['main_emotion'] = X_data['main_emotion'].astype(str)

/tmp/ipython-input-252-998666315.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_data.replace({'main_emotion':['anger','annoyance','confusion','disappointment','disapproval','disgust','embarrassment','excitement','fear','nervousness','remorse','sadness']},-1,inplace=True)


In [ ]:
print(np.unique(X_data['main_emotion']))

['-1' '0' '1']


In [ ]:
#Checking the dsitribution of the target column
X_data['main_emotion'].value_counts()

,count
main_emotion,
0,2848
1,1433
-1,719


In [ ]:
port_stem = PorterStemmer()

def stemming(content):
        stemmed_content = re.sub('[^a-zA-Z]',' ', content)
        stemmed_content = stemmed_content.lower()
        stemmed_content = stemmed_content.split()
        stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
        stemmed_content = ' '.join(stemmed_content)
        return stemmed_content

X_data['stemmed_content']= X_data['original_text'].apply(stemming)

X_data.head()
print(X_data['stemmed_content'])
print(X_data['main_emotion'])

#seperating the data and label
X= X_data['stemmed_content'].values
Y= X_data['main_emotion'].values

print(X)
print(Y)

#splitting the data into training and test

X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size =.2,stratify=Y,random_state=2)

print(X.shape,X_train.shape,X_test.shape)

print(X_train)
print(X_test)








0       make us realiz option see option deliv saturda...
1       grok ai repres scenario yeah miss tusk varahav...
2                                   pretti sure may undef
3       check r underr program languag ideal data anal...
4                     came car know go home smh piss cold
                              ...                        
4995                think enough cat health mainten parvo
4996                        sure mom ass get pretti messi
4997    joo bayden fodii wa ude njillu jaango altin an...
4998                                wing u dont mind give
4999    abbydphillip done play realli bad refere night...
Name: stemmed_content, Length: 5000, dtype: object
0        1
1        0
2        1
3        0
4        0
        ..
4995    -1
4996    -1
4997     0
4998     0
4999     0
Name: main_emotion, Length: 5000, dtype: object
['make us realiz option see option deliv saturday adapt dumb'
 'grok ai repres scenario yeah miss tusk varahavtar look badass say least'
 'pre

In [ ]:
#COnverting the textual data to numerical data
vectorizer= TfidfVectorizer()

X_train= vectorizer.fit_transform(X_train)
X_test =vectorizer.transform(X_test)


print(X_train)
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 41474 stored elements and shape (4000, 10718)>
  Coords	Values
  (0, 5426)	0.10976981413640777
  (0, 7285)	0.15768980052858078
  (0, 2391)	0.16303236500154852
  (0, 9615)	0.17593716793629063
  (0, 742)	0.13086029694698573
  (0, 4712)	0.15080203697575278
  (0, 5144)	0.4118623291541667
  (0, 7732)	0.4118623291541667
  (0, 3997)	0.1651156153966909
  (0, 8654)	0.1673975580185404
  (0, 8142)	0.1651156153966909
  (0, 7452)	0.15614460144872053
  (0, 7722)	0.12531043874698145
  (0, 3110)	0.183993079061328
  (0, 6758)	0.1520301151822815
  (0, 1732)	0.3222319208197868
  (0, 10398)	0.17593716793629063
  (0, 7724)	0.20593116457708335
  (0, 5146)	0.20593116457708335
  (0, 5145)	0.20593116457708335
  (0, 5147)	0.20593116457708335
  (0, 7480)	0.20593116457708335
  (1, 6248)	0.41417403184356316
  (1, 10379)	0.2049330423776125
  (1, 5590)	0.21585787602852255
  :	:
  (3997, 7625)	0.394631194151923
  (3997, 206)	0.4266365241228009
  (3997, 848

In [ ]:
#Training the logistical Model


#logistic regression

model = LogisticRegression(max_iter=10000)
model.fit(X_train,Y_train)







LogisticRegression(max_iter=10000)

In [ ]:
#Model Eval
#AccuracyScore

#Accuaracy score on training data
X_train_prediction= model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)




In [ ]:
print("Accuracy Score on the training data : ",training_data_accuracy)

Accuracy Score on the training data :  0.8065


In [ ]:
#Accuaracy score on test data
X_test_prediction= model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)

print("Accuracy Score on the testing data : ",test_data_accuracy)


Accuracy Score on the testing data :  0.681


In [ ]:
import pickle

filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

#loading the saved model

loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

X_new= X_test[200]
print(Y_test[200])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['1']
Positive Tweet


In [ ]:
X_new= X_test[15]
print(Y_test[15])


predicition = model.predict(X_new)
print (predicition)

if (predicition[0]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['0']
Positive Tweet


In [ ]:
X_new= X_test[3]
print(Y_test[3])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['0']
Positive Tweet


In [ ]:
X_new= X_test[17]
print(Y_test[17])


predicition = model.predict(X_new)
print (predicition)

if (predicition[-1]==-1):
        print('Negative Tweet')

else:
        print('Positive Tweet')

0
['0']
Positive Tweet


In [ ]:
#Logistic regression Model

In [ ]:
#Train Accuracy 80%

In [ ]:
#Test Accurary 68%
#Metrics per class:
#Class -1 - Precision: 0.522, Recall: 0.083, F1-Score: 0.144
#Class 0 - Precision: 0.678, Recall: 0.937, F1-Score: 0.786
#Class 1 - Precision: 0.720, Recall: 0.476, F1-Score: 0.573

#Macro F1 Score: 0.5009316055123301
#Weighted F1 Score: 0.6327441201188044
#Overall Accuracy: 0.682
#Data
#5k rows of English language, x.com data from Hugging Face
#Pre-processed in dataset_cleaning.ipynb file.

In [ ]:
#F1 and other metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np


# Assuming you have a trained model and predictions:
# model is the trained XGBClassifier from cell L9QaEvPBilZw

# Predict using the trained model from cell L9QaEvPBilZw
# Ensure 'model' here refers to the trained model from L9QaEvPBilZw (which is XGBClassifier)
Y_pred = model.predict(X_test)

# Calculate and print per-class precision, recall, and F1-score
# Use zero_division=1 to handle cases where a class has no predicted samples
precision_per_class = precision_score(Y_test, Y_pred, average=None, zero_division=1)
recall_per_class = recall_score(Y_test, Y_pred, average=None, zero_division=1)
f1_per_class = f1_score(Y_test, Y_pred, average=None, zero_division=1)

# Get the unique labels
unique_labels = np.unique(Y_test)

print("Metrics per class:")
for i, label in enumerate(unique_labels):
    print(f"Class {label} - Precision: {precision_per_class[i]:.3f}, Recall: {recall_per_class[i]:.3f}, F1-Score: {f1_per_class[i]:.3f}")

# Optionally print macro and weighted F1 scores (already working)
print("\nMacro F1 Score:", f1_score(Y_test, Y_pred, average='macro'))
print("Weighted F1 Score:", f1_score(Y_test, Y_pred, average='weighted'))

# Also print overall accuracy
print("Overall Accuracy:", accuracy_score(Y_test, Y_pred))

Metrics per class:
Class -1 - Precision: 0.667, Recall: 0.111, F1-Score: 0.190
Class 0 - Precision: 0.665, Recall: 0.953, F1-Score: 0.783
Class 1 - Precision: 0.767, Recall: 0.427, F1-Score: 0.548

Macro F1 Score: 0.5072585522088532
Weighted F1 Score: 0.630547918819675
Overall Accuracy: 0.681
